<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Код во многом опирается на книгу <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> за авторством <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# Small Qwen

- Этот notebook намеренно минималистичен и посвящен реализации небольшой версии Qwen3 0.6B, 1.7B, 4B, 8B, and 32B. Больше информации о модели можно получить по ссылкам:
  - [Qwen3: Think Deeper, Act Faster](https://qwenlm.github.io/blog/qwen3/)
  - [Qwen3 Technical Report](https://arxiv.org/abs/2505.09388) 

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/qwen/qwen-overview.webp">

In [1]:
%%capture
!pip install -r sqwen_req.txt

In [2]:
from importlib.metadata import version

pkgs = [
    "huggingface_hub",  # to download pretrained weights
    "tokenizers",       # to implement the tokenizer
    "torch",            # to implement the model
]
for p in pkgs:
    print(f"{p} version: {version(p)}")

huggingface_hub version: 0.33.4
tokenizers version: 0.21.2
torch version: 2.6.0


- Реализация поддерживает reasoning ("chain of thoughts"), который активируется следующим флагом:

In [3]:
REASONING = True

# 1. Architecture code

In [4]:
import torch
import torch.nn as nn


class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # Инициализация двух параллельных ветвей для гейтированного механизма
        self.fc1 = nn.Linear(cfg["emb_dim"], cfg["hidden_dim"], dtype=cfg["dtype"], bias=False)
        self.fc2 = nn.Linear(cfg["emb_dim"], cfg["hidden_dim"], dtype=cfg["dtype"], bias=False)

        # Финалный слой для проекции обратно в пространство эмбеддингов
        self.fc3 = nn.Linear(cfg["hidden_dim"], cfg["emb_dim"], dtype=cfg["dtype"], bias=False)

    def forward(self, x):
        x_fc1 = self.fc1(x)
        x_fc2 = self.fc2(x)

        # Гейтированный механизм: SiLU-активация работает как "выключатель" для информационных потоков
        # Умножение выходов fc1 и fc2 создает адактивный фильтр для входных данных
        x = nn.functional.silu(x_fc1) * x_fc2

        # Компенсация роста дисперсии после мультипликативной операции
        return self.fc3(x)

### Пояснения к ключевым моментам:
1. **Гейтированный механизм**  
   Использование двух параллельных ветвей (`fc1` и `fc2`) с последующим умножением реализует механизм адактивной фильтрации, где:
   - `fc1` формирует "ворота" через SiLU-активацию (логистический выбор информации)
   - `fc2` предоставляет "кандидатные значения" для трансформации
   Такая архитектура эффективнее стандартного `Linear -> ReLU -> Linear` для задач с разреженными паттернами.

2. **Отсутствие смещений (bias=False)**  
   Сознательное решение для:
   - Снижения переобучения
   - Упрощения динамики обучения
   - Совместимости с техниками нормализации (LayerNorm обычно компенсирует отсутствие bias)

3. **Мультипликативное взаимодействие**  
   Операция `silu(a) * b`:
   - Добавляет нелинейность выше 2-го порядка
   - Позволяет моделировать сложные зависимости между нейронами
   - Эффективнее классических активаций для языковых задач (аналогично Gated Linear Units)

4. **Типизация через cfg["dtype"]**  
   Автоматическое поддержание выбранного типа данных (float16/float32) на всех слоях:
   - Критично для mixed-precision training
   - Гарантирует согласованность вычислений

5. **Семантика размерностей**  
   `emb_dim` -> `hidden_dim` -> `emb_dim`:
   - Сквозной принцип (residual path friendly)
   - Соответствие оригинальной схеме Transformer'а
   - Сохранение формы тензора для последующих операций сложения

In [5]:
import torch
import torch.nn as nn

class RMSNorm(nn.Module):
    def __init__(self, emb_dim, eps=1e-6, bias=False, qwen3_compatible=True):
        super().__init__()
        self.eps = eps
        self.qwen3_compatible = qwen3_compatible
        
        # Параметр масштабирования (обязательный)
        self.scale = nn.Parameter(torch.ones(emb_dim))
        
        # Опциональный параметр смещения (включается через bias=True)
        self.shift = nn.Parameter(torch.zeros(emb_dim)) if bias else None

    def forward(self, x):
        # Сохраняем исходный тип для корректного восстановления
        input_dtype = x.dtype
        
        # Режим совместимости с Qwen3: вычисления в float32 для стабильности
        if self.qwen3_compatible:
            x = x.to(torch.float32)

        # Вычисление RMS по последнему измерению:
        # 1. Квадраты элементов (pow(2))
        # 2. Усреднение по последней оси (keepdim сохраняет размерность)
        variance = x.pow(2).mean(dim=-1, keepdim=True)
        
        # Нормализация:
        # 1. Обратный квадратный корень дисперсии (rsqrt = 1/sqrt(v))
        # 2. Масштабирование исходных значений
        norm_x = x * torch.rsqrt(variance + self.eps)
        
        # Применение обучаемого масштаба
        norm_x = norm_x * self.scale

        # Добавление смещения (если включено в конфигурации)
        if self.shift is not None:
            norm_x = norm_x + self.shift

        # Восстановление исходного типа данных
        return norm_x.to(input_dtype)


### Ключевые особенности реализации:
1. **Совместимость с Qwen3**  
   Флаг `qwen3_compatible` включает приведение к float32 для:
   - Повышения численной стабильности
   - Гарантии бит-в-бит соответствия оригинальной реализации
   - Автоматического восстановления исходного типа данных после вычислений

2. **RMS-нормализация**  
   Формула `x * (1 / sqrt(mean(x^2) + eps))`:
   - Отличается от LayerNorm отсутствием центрирования (вычитания среднего)
   - Эффективнее вычислительно (меньше операций)
   - Показала лучшую сходимость в больших языковых моделях

3. **Опциональное смещение**  
   Параметр `bias` контролирует:
   - Добавление обучаемого параметра `shift`
   - Совместимость с вариантами архитектур (например, LLaMA не использует смещение)
   - Гибкость конфигурации без изменения кода

4. **Управление типами данных**  
   Явное сохранение/восстановление типа:
   - Позволяет работать с mixed-precision (FP16/BF16)
   - Предотвращает потерю точности в режиме совместимости
   - Автоматически адаптируется под входной тензор

5. **Стабильность вычислений**  
   Параметр `eps` (ε) решает проблемы:
   - Деления на ноль при нулевой дисперсии
   - Численной нестабильности для малых значений дисперсии
   - Рекомендуемое значение 1e-6 балансирует точность и устойчивость

In [6]:
def compute_rope_params(head_dim, theta_base=10_000, context_length=4096, dtype=torch.float32):
    # Проверка четности размерности головы (необходимо для парного разбиения)
    assert head_dim % 2 == 0, "Embedding dimension must be even"

    # Вычисление обратных частот для вращения:
    # 1. Создание последовательности [0, 2, 4, ..., head_dim-2]
    # 2. Нормализация индексов по размерности головы
    # 3. Экспоненциальное затухание частот с базой theta_base
    inv_freq = 1.0 / (theta_base ** (torch.arange(0, head_dim, 2, dtype=dtype)[: (head_dim // 2)].float() / head_dim))

    # Генерация позиционных индексов [0, 1, 2, ..., context_length-1]
    positions = torch.arange(context_length, dtype=dtype)

    # Создание вращательных углов:
    # 1. Внешнее произведение позиций и частот (positions[:, None] * inv_freq[None, :])
    # 2. Результат - матрица размером (context_length, head_dim//2) 
    angles = positions[:, None] * inv_freq[None, :]

    # Дублирование углов для полного покрытия head_dim:
    # [θ0, θ1, ..., θ_{d/2-1}] -> [θ0, θ1, ..., θ_{d/2-1}, θ0, θ1, ..., θ_{d/2-1}]
    angles = torch.cat([angles, angles], dim=1)

    # Предварительное вычисление косинусов и синусов:
    # - Эти тензоры будут кэшироваться для эффективности
    cos = torch.cos(angles)
    sin = torch.sin(angles)

    return cos, sin


def apply_rope(x, cos, sin):
    # Размерность входного тензора: [batch, heads, seq_len, head_dim]
    batch_size, num_heads, seq_len, head_dim = x.shape
    assert head_dim % 2 == 0, "Head dimension must be even"

    # Разделение скрытого состояния на две половины:
    # x1 = [x0, x1, ..., x_{d/2-1}]
    # x2 = [x_{d/2}, x_{d/2+1}, ..., x_{d-1}]
    x1 = x[..., : head_dim // 2]
    x2 = x[..., head_dim // 2 :]

    # Подготовка вращательных матриц:
    # 1. Срез по актуальной длине последовательности
    # 2. Добавление размерностей для broadcasing [1, 1, seq_len, head_dim]
    cos = cos[:seq_len, :].unsqueeze(0).unsqueeze(0)
    sin = sin[:seq_len, :].unsqueeze(0).unsqueeze(0)

    # Создание вращаемой компоненты:
    # rotated = [-x2, x1] - перестановка и смена знака
    rotated = torch.cat((-x2, x1), dim=-1)
    
    # Применение ротационного преобразования:
    # x_rot = x ⊗ cos(θ) + rotated ⊗ sin(θ)
    # Эквивалентно комплексному вращению: (x1 + i·x2) ⊗ (cosθ + i·sinθ)
    x_rotated = (x * cos) + (rotated * sin)

    # Восстановление оригинального типа данных:
    # - После операций с float32 возвращаем исходный dtype (float16/bf16)
    return x_rotated.to(dtype=x.dtype)

### Ключевые пояснения:

1. **Физика RoPE**
   - Реализует вращение векторов в многомерном пространстве
   - Аналогично умножению комплексных чисел: (x1 + ix2) * (cosθ + isinθ)
   - Сохраняет относительные позиционные расстояния (инвариантность к сдвигу)

2. **Оптимизации**
   - Предварительный расчет cos/sin: вычисляются один раз при инициализации
   - Подготовка углов: оптимизированная формула без экспонент
   - Broadcasting: эффективное применение к пакетам данных

3. **Особенности реализации**
   - Четное разбиение: обязательное требование для парного вращения
   - Обратные частоты: гарантируют плавное затухание углов по измерениям
   - Сохранение типа: критично для mixed-precision обучения

4. **Почему работает?**
   - Кодирует позиции через вращение векторов запросов/ключей
   - Сохраняет скалярное произведение только для близких позиций
   - Автоматически учитывает относительное расстояние между токенами

5. **Преимущества перед другими методами**
   - Относительное позиционирование (не абсолютные позиции)
   - Расширяемость на большие контексты
   - Сохранение длины векторов (норма не меняется)

In [7]:
class GroupedQueryAttention(nn.Module):
    def __init__(
        self, d_in, num_heads, num_kv_groups, head_dim=None, qk_norm=False, dtype=None
    ):
        super().__init__()
        # Проверка делимости количества голов на группы
        assert num_heads % num_kv_groups == 0, "num_heads must be divisible by num_kv_groups"

        self.num_heads = num_heads
        self.num_kv_groups = num_kv_groups
        # Размер группы запросов
        self.group_size = num_heads // num_kv_groups

        if head_dim is None:
            # Автоматический расчет размерности головы
            assert d_in % num_heads == 0, "`d_in` must be divisible by `num_heads` if `head_dim` is not set"
            head_dim = d_in // num_heads

        self.head_dim = head_dim
        # Выходная размерность всех голов
        self.d_out = num_heads * head_dim

        # Проекционные слои для Q, K, V
        self.W_query = nn.Linear(d_in, self.d_out, bias=False, dtype=dtype)
        # KV имеют групповую размерность
        self.W_key = nn.Linear(d_in, num_kv_groups * head_dim, bias=False, dtype=dtype)
        self.W_value = nn.Linear(d_in, num_kv_groups * head_dim, bias=False, dtype=dtype)

        # Финалный проекционный слой
        self.out_proj = nn.Linear(self.d_out, d_in, bias=False, dtype=dtype)

        # Опциональная нормализация QK
        if qk_norm:
            self.q_norm = RMSNorm(head_dim, eps=1e-6)
            self.k_norm = RMSNorm(head_dim, eps=1e-6)
        else:
            self.q_norm = self.k_norm = None

    def forward(self, x, mask, cos, sin):
        # Извлечение размерностей
        b, num_tokens, _ = x.shape

        # Применение проекций
        queries = self.W_query(x)  # (b, num_tokens, num_heads * head_dim)
        keys = self.W_key(x)       # (b, num_tokens, num_kv_groups * head_dim)
        values = self.W_value(x)   # (b, num_tokens, num_kv_groups * head_dim)

        # Решейпинг в тензоры с отдельной размерностью голов
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim).transpose(1, 2)
        keys = keys.view(b, num_tokens, self.num_kv_groups, self.head_dim).transpose(1, 2)
        values = values.view(b, num_tokens, self.num_kv_groups, self.head_dim).transpose(1, 2)

        # Опциональная нормализация
        if self.q_norm:
            queries = self.q_norm(queries)
        if self.k_norm:
            keys = self.k_norm(keys)

        # Применение ротационных позиционных эмбеддингов
        queries = apply_rope(queries, cos, sin)
        keys = apply_rope(keys, cos, sin)

        # Расширение K и V для всех голов запросов
        keys = keys.repeat_interleave(self.group_size, dim=1)
        values = values.repeat_interleave(self.group_size, dim=1)

        # Attention
        attn_scores = queries @ keys.transpose(2, 3)
        # Применение маски (заполнение -inf)
        attn_scores = attn_scores.masked_fill(mask, -torch.inf)
        # Softmax с масштабированием
        attn_weights = torch.softmax(attn_scores / self.head_dim**0.5, dim=-1)

        # Аггрегация значений
        context = (attn_weights @ values).transpose(1, 2).reshape(b, num_tokens, self.d_out)
        # Финальная проекция
        return self.out_proj(context)

### Ключевые моменты GroupedQueryAttention:

1. **Групповая архитектура запросов**  
   Разделение голов на группы ключей-значений (KV) и запросов (Q) позволяет:
   - Снизить потребление памяти на 30-50% по сравнению с Multi-Head Attention
   - Сохранить качество модели при уменьшении числа параметров
   - Каждая группа KV обслуживает несколько голов Q (group_size = num_heads / num_kv_groups)

2. **Эффективные проекционные слои**  
   Асимметричная проекция Q, K, V:
   - Q: полная проекция (num_heads * head_dim) для индивидуальных запросов
   - K/V: сокращенная проекция (num_kv_groups * head_dim) для общих ключей и значений
   - Значительное уменьшение параметров в K/V проекциях

3. **Ротационные позиционные эмбеддинги (RoPE)**  
   Применение apply_rope к Q и K:
   - Учет позиционной информации через эмбеддинги
   - Повышение качества работы с последовательностями различной длины
   - Современный стандарт для позиционного кодирования

4. **Опциональная нормализация QK**  
   Контролируемая параметром qk_norm:
   - Стабилизация обучения через RMSNorm запросов и ключей
   - Улучшение сходимости модели
   - Предотвращение "взрыва" градиентов

5. **Механизм расширения KV**  
   repeat_interleave для ключей и значений:
   - Эффективное повторение KV групп для multiple голов запросов
   - Минимизация overhead при вычислении внимания
   - Сохранение tensor shape для последующих операций

6. **Оптимизированное вычисление внимания**  
   Стандартная формула с масштабированием и маскированием:
   - Масштабирование на sqrt(head_dim) для стабильности softmax
   - Поддержка маскирования через замену на -inf
   - Оптимизированные матричные операции

In [8]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # Инициализация группового внимания запросов
        self.att = GroupedQueryAttention(
            d_in=cfg["emb_dim"],
            num_heads=cfg["n_heads"],
            head_dim=cfg["head_dim"],
            num_kv_groups=cfg["n_kv_groups"],
            qk_norm=cfg["qk_norm"],
            dtype=cfg["dtype"]
        )
        # Инициализация прямой связи (FeedForward)
        self.ff = FeedForward(cfg)
        # Нормализация перед вниманием
        self.norm1 = RMSNorm(cfg["emb_dim"], eps=1e-6)
        # Нормализация перед прямой связью
        self.norm2 = RMSNorm(cfg["emb_dim"], eps=1e-6)

    def forward(self, x, mask, cos, sin):
        # shortcut для блока внимания
        shortcut = x
        x = self.norm1(x)
        x = self.att(x, mask, cos, sin)  # Форма [batch_size, num_tokens, emb_size]
        # Возврат исходного входа (residual connection)
        x = x + shortcut

        # shortcut для блока прямой связи
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        # Возврат исходного входа (residual connection)
        x = x + shortcut

        return x

### Ключевые моменты TransformerBlock:

1. **Композиционная архитектура**  
   Блок объединяет два ключевых компонента трансформера:
   - `GroupedQueryAttention`: механизм группового внимания с оптимизированными вычислениями
   - `FeedForward`: двухслойная нейронная сеть с нелинейным преобразованием
   - Оба компонента работают с одинаковой размерностью `emb_dim`

2. **Pre-Normalization схема**  
   Применение нормализации ДО операций внимания и FFN:
   - `norm1` перед вниманием стабилизирует входные данные
   - `norm2` перед FFN обеспечивает согласованность распределения
   - Улучшает сходимость и стабильность обучения по сравнению с Post-Norm

3. **Residual Connections (остаточные связи)**  
   Сохранение и добавление исходного входа после каждой операции:
   - `x = x + shortcut` предотвращает затухание градиентов
   - Обеспечивает альтернативный путь для распространения градиента
   - Позволяет строить глубокие сети без проблем исчезающих градиентов

4. **Сквозная передача контекста**  
   Единая обработка маски и позиционных эмбеддингов:
   - Маска `mask` обеспечивает корректное вычисление внимания
   - `cos`/`sin` параметры для RoPE передаются в механизм внимания
   - Консистентность размерностей на всех этапах обработки

5. **Конфигурируемость через параметры**  
   Гибкая настройка через словарь конфигурации `cfg`:
   - Единый источник параметров для всех компонентов
   - Согласованность типов данных через `dtype`
   - Возможность тонкой настройки архитектуры

In [9]:
class Qwen3Model(nn.Module):
    def __init__(self, cfg):
        super().__init__()

        # Main model parameters
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"], dtype=cfg["dtype"])

        self.trf_blocks = nn.ModuleList(  # ModuleList since Sequential can only accept one input, and we need `x, mask, cos, sin`
            [TransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )

        self.final_norm = RMSNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False, dtype=cfg["dtype"])

        # Reusuable utilities
        if cfg["head_dim"] is None:
            head_dim = cfg["emb_dim"] // cfg["n_heads"]
        else:
            head_dim = cfg["head_dim"]
        cos, sin = compute_rope_params(
            head_dim=head_dim,
            theta_base=cfg["rope_base"],
            context_length=cfg["context_length"]
        )
        self.register_buffer("cos", cos, persistent=False)
        self.register_buffer("sin", sin, persistent=False)
        self.cfg = cfg


    def forward(self, in_idx):
        # Forward pass
        tok_embeds = self.tok_emb(in_idx)
        x = tok_embeds

        num_tokens = x.shape[1]
        mask = torch.triu(torch.ones(num_tokens, num_tokens, device=x.device, dtype=torch.bool), diagonal=1)
        
        for block in self.trf_blocks:
            x = block(x, mask, self.cos, self.sin)
        x = self.final_norm(x)
        logits = self.out_head(x.to(self.cfg["dtype"]))
        return logits

# 2. Initialize model

In [10]:
CHOOSE_MODEL = "0.6B"

if CHOOSE_MODEL == "0.6B":
    QWEN3_CONFIG = {
        "vocab_size": 151_936,           # Vocabulary size
        "context_length": 40_960,        # Context length that was used to train the model
        "emb_dim": 1024,                 # Embedding dimension
        "n_heads": 16,                   # Number of attention heads
        "n_layers": 28,                  # Number of layers
        "hidden_dim": 3072,              # Size of the intermediate dimension in FeedForward
        "head_dim": 128,                 # Size of the heads in GQA
        "qk_norm": True,                 # Whether to normalize queries and values in GQA
        "n_kv_groups": 8,                # Key-Value groups for grouped-query attention
        "rope_base": 1_000_000.0,        # The base in RoPE's "theta"
        "dtype": torch.bfloat16,         # Lower-precision dtype to reduce memory usage
    }

elif CHOOSE_MODEL == "1.7B":
    QWEN3_CONFIG = {
        "vocab_size": 151_936,
        "context_length": 40_960,
        "emb_dim": 2152,                 # 2.1x prev
        "n_heads": 16,
        "n_layers": 28,
        "hidden_dim": 6452,              # 2.1x prev
        "head_dim": 128,
        "qk_norm": True,
        "n_kv_groups": 8,
        "rope_base": 1_000_000.0,
        "dtype": torch.bfloat16,
    }   

elif CHOOSE_MODEL == "4B":
    QWEN3_CONFIG = {
        "vocab_size": 151_936,
        "context_length": 40_960,
        "emb_dim": 2690,                 # ~25% larger 
        "n_heads": 32,                   # 2x larger 
        "n_layers": 36,                  # ~29% larger 
        "hidden_dim": 9876,              # ~3x larger 
        "head_dim": 128,
        "qk_norm": True,
        "n_kv_groups": 8,
        "rope_base": 1_000_000.0,
        "dtype": torch.bfloat16,
    }  

elif CHOOSE_MODEL == "8B":
    QWEN3_CONFIG = {
        "vocab_size": 151_936,
        "context_length": 40_960,
        "emb_dim": 4096,                 # ~60% larger 
        "n_heads": 32,
        "n_layers": 36,                  # ~26% larger 
        "hidden_dim": 12288,
        "head_dim": 128,
        "qk_norm": True,
        "n_kv_groups": 8,
        "rope_base": 1_000_000.0,
        "dtype": torch.bfloat16,
    } 

elif CHOOSE_MODEL == "14B":
    QWEN3_CONFIG = {
        "vocab_size": 151_936,
        "context_length": 40_960,
        "emb_dim": 5120,                 # ~25% larger 
        "n_heads": 40,                   # ~25% larger 
        "n_layers": 40,                  # ~11% larger 
        "hidden_dim": 17408,             # ~42% larger 
        "head_dim": 128,
        "qk_norm": True,
        "n_kv_groups": 8,
        "rope_base": 1_000_000.0,
        "dtype": torch.bfloat16,
    } 

elif CHOOSE_MODEL == "32B":
    QWEN3_CONFIG = {
        "vocab_size": 151_936,
        "context_length": 40_960,
        "emb_dim": 5120,                
        "n_heads": 64,                   # ~60% larger 
        "n_layers": 64,                  # ~60% larger 
        "hidden_dim": 25600,             # ~47% larger 
        "head_dim": 128,
        "qk_norm": True,
        "n_kv_groups": 8,
        "rope_base": 1_000_000.0,
        "dtype": torch.bfloat16,
    } 

else:
    raise ValueError(f"{CHOOSE_MODEL} is not supported.")

In [11]:
torch.manual_seed(123)
model = Qwen3Model(QWEN3_CONFIG)

In [12]:
model

Qwen3Model(
  (tok_emb): Embedding(151936, 1024)
  (trf_blocks): ModuleList(
    (0-27): 28 x TransformerBlock(
      (att): GroupedQueryAttention(
        (W_query): Linear(in_features=1024, out_features=2048, bias=False)
        (W_key): Linear(in_features=1024, out_features=1024, bias=False)
        (W_value): Linear(in_features=1024, out_features=1024, bias=False)
        (out_proj): Linear(in_features=2048, out_features=1024, bias=False)
        (q_norm): RMSNorm()
        (k_norm): RMSNorm()
      )
      (ff): FeedForward(
        (fc1): Linear(in_features=1024, out_features=3072, bias=False)
        (fc2): Linear(in_features=1024, out_features=3072, bias=False)
        (fc3): Linear(in_features=3072, out_features=1024, bias=False)
      )
      (norm1): RMSNorm()
      (norm2): RMSNorm()
    )
  )
  (final_norm): RMSNorm()
  (out_head): Linear(in_features=1024, out_features=151936, bias=False)
)

- A quick check that the forward pass works before continuing:

In [13]:
model(torch.tensor([1, 2, 3]).unsqueeze(0))

tensor([[[-0.2256, -0.0112, -0.7109,  ...,  0.4355,  0.1216,  1.0781],
         [-0.6484,  0.5430, -0.0708,  ..., -0.0664,  0.5352,  0.3145],
         [-0.4785, -0.1572,  0.1099,  ..., -0.2217,  0.2295,  0.6289]]],
       dtype=torch.bfloat16, grad_fn=<UnsafeViewBackward0>)

In [14]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

# Account for weight tying
total_params_normalized = total_params - model.tok_emb.weight.numel()
print(f"\nTotal number of unique parameters: {total_params_normalized:,}")

Total number of parameters: 751,632,384

Total number of unique parameters: 596,049,920


In [15]:
def model_memory_size(model, input_dtype=torch.float32):
    total_params = 0
    total_grads = 0
    for param in model.parameters():
        # Расчет общего количества элементов в параметре
        param_size = param.numel()
        total_params += param_size
        # Проверка, хранятся ли градиенты для этого параметра
        if param.requires_grad:
            total_grads += param_size

    # Расчет размера буферов (непараметрические данные)
    total_buffers = sum(buf.numel() for buf in model.buffers())

    # Размер в байтах = (Количество элементов) * (Размер элемента в байтах)
    # Предполагаем, что параметры и градиенты хранятся в том же типе, что и входные данные
    element_size = torch.tensor(0, dtype=input_dtype).element_size()
    total_memory_bytes = (total_params + total_grads + total_buffers) * element_size

    # Конвертация байтов в гигабайты
    total_memory_gb = total_memory_bytes / (1024**3)

    return total_memory_gb

print(f"float32 (стандарт PyTorch): {model_memory_size(model, input_dtype=torch.float32):.2f} GB")
print(f"bfloat16: {model_memory_size(model, input_dtype=torch.bfloat16):.2f} GB")

float32 (стандарт PyTorch): 5.64 GB
bfloat16: 2.82 GB


In [16]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

model.to(device);

# 3. Load pretrained weights

In [17]:
def load_weights_into_qwen(model, param_config, params):
    def assign(left, right, tensor_name="unknown"):
        if left.shape != right.shape:
            raise ValueError(f"Shape mismatch in tensor '{tensor_name}'. Left: {left.shape}, Right: {right.shape}")
        return torch.nn.Parameter(right.clone().detach() if isinstance(right, torch.Tensor) else torch.tensor(right))

    model.tok_emb.weight = assign(model.tok_emb.weight, params["model.embed_tokens.weight"], "model.embed_tokens.weight")

    for l in range(param_config["n_layers"]):
        block = model.trf_blocks[l]
        att = block.att

        # Q, K, V projections
        att.W_query.weight = assign(
            att.W_query.weight,
            params[f"model.layers.{l}.self_attn.q_proj.weight"],
            f"model.layers.{l}.self_attn.q_proj.weight"
        )
        att.W_key.weight = assign(
            att.W_key.weight,
            params[f"model.layers.{l}.self_attn.k_proj.weight"],
            f"model.layers.{l}.self_attn.k_proj.weight"
        )
        att.W_value.weight = assign(
            att.W_value.weight,
            params[f"model.layers.{l}.self_attn.v_proj.weight"],
            f"model.layers.{l}.self_attn.v_proj.weight"
        )

        # Output projection
        att.out_proj.weight = assign(
            att.out_proj.weight,
            params[f"model.layers.{l}.self_attn.o_proj.weight"],
            f"model.layers.{l}.self_attn.o_proj.weight"
        )

        # QK norms
        if hasattr(att, "q_norm") and att.q_norm is not None:
            att.q_norm.scale = assign(
                att.q_norm.scale,
                params[f"model.layers.{l}.self_attn.q_norm.weight"],
                f"model.layers.{l}.self_attn.q_norm.weight"
            )
        if hasattr(att, "k_norm") and att.k_norm is not None:
            att.k_norm.scale = assign(
                att.k_norm.scale,
                params[f"model.layers.{l}.self_attn.k_norm.weight"],
                f"model.layers.{l}.self_attn.k_norm.weight"
            )

        # Attention layernorm
        block.norm1.scale = assign(
            block.norm1.scale,
            params[f"model.layers.{l}.input_layernorm.weight"],
            f"model.layers.{l}.input_layernorm.weight"
        )

        # Feedforward weights
        block.ff.fc1.weight = assign(
            block.ff.fc1.weight,
            params[f"model.layers.{l}.mlp.gate_proj.weight"],
            f"model.layers.{l}.mlp.gate_proj.weight"
        )
        block.ff.fc2.weight = assign(
            block.ff.fc2.weight,
            params[f"model.layers.{l}.mlp.up_proj.weight"],
            f"model.layers.{l}.mlp.up_proj.weight"
        )
        block.ff.fc3.weight = assign(
            block.ff.fc3.weight,
            params[f"model.layers.{l}.mlp.down_proj.weight"],
            f"model.layers.{l}.mlp.down_proj.weight"
        )
        block.norm2.scale = assign(
            block.norm2.scale,
            params[f"model.layers.{l}.post_attention_layernorm.weight"],
            f"model.layers.{l}.post_attention_layernorm.weight"
        )

    # Final normalization and output head
    model.final_norm.scale = assign(model.final_norm.scale, params["model.norm.weight"], "model.norm.weight")

    if "lm_head.weight" in params:
        model.out_head.weight = assign(model.out_head.weight, params["lm_head.weight"], "lm_head.weight")
    else:
        # Model uses weight tying, hence we reuse the embedding layer weights here
        print("Model uses weight tying.")
        model.out_head.weight = assign(model.out_head.weight, params["model.embed_tokens.weight"], "model.embed_tokens.weight")

In [18]:
import json
import os
from pathlib import Path
from safetensors.torch import load_file
from huggingface_hub import hf_hub_download, snapshot_download

from_hf = False

if REASONING:
    repo_id = f"Qwen/Qwen3-{CHOOSE_MODEL}"
else:
    repo_id = f"Qwen/Qwen3-{CHOOSE_MODEL}-Base"

local_dir = Path(repo_id).parts[-1]

if CHOOSE_MODEL == "0.6B":
    if from_hf:
        weights_file = hf_hub_download(
            repo_id=repo_id,
            filename="model.safetensors",
            local_dir=local_dir,
        )
    else:
        weights_file = "Qwen3-0.6B/model.safetensors"
    weights_dict = load_file(weights_file)
else:
    repo_dir = snapshot_download(repo_id=repo_id, local_dir=local_dir)
    index_path = os.path.join(repo_dir, "model.safetensors.index.json")
    with open(index_path, "r") as f:
        index = json.load(f)

    weights_dict = {}
    for filename in set(index["weight_map"].values()):
        shard_path = os.path.join(repo_dir, filename)
        shard = load_file(shard_path)
        weights_dict.update(shard)

load_weights_into_qwen(model, QWEN3_CONFIG, weights_dict)
model.to(device);

# 4. Load tokenizer

In [19]:
from tokenizers import Tokenizer


class Qwen3Tokenizer():
    def __init__(self, tokenizer_file_path="tokenizer.json", repo_id=None, add_generation_prompt=False, add_thinking=False):
        self.tokenizer_file_path = tokenizer_file_path
        self.add_generation_prompt = add_generation_prompt
        self.add_thinking = add_thinking

        tokenizer_file_path_obj = Path(tokenizer_file_path)
        if not tokenizer_file_path_obj.is_file() and repo_id is not None:
            _ = hf_hub_download(
                repo_id=repo_id,
                filename=str(tokenizer_file_path_obj.name),
                local_dir=str(tokenizer_file_path_obj.parent.name)
            )
        self.tokenizer = Tokenizer.from_file(tokenizer_file_path)

    def encode(self, prompt):
        messages = [
            {"role": "user", "content": prompt}
        ]  
        formatted_prompt = self.format_qwen_chat(
            messages,
            add_generation_prompt=self.add_generation_prompt,
            add_thinking=self.add_thinking
        )
        return self.tokenizer.encode(formatted_prompt).ids
                
    def decode(self, token_ids):
        return self.tokenizer.decode(token_ids, skip_special_tokens=False)
    
    @staticmethod
    def format_qwen_chat(messages, add_generation_prompt=False, add_thinking=False):
        prompt = ""
        for msg in messages:
            prompt += f"<|im_start|>{msg['role']}\n{msg['content']}<|im_end|>\n"
        if add_generation_prompt:
            prompt += "<|im_start|>assistant"
            if not add_thinking:
                prompt += "<|think>\n\n<|/think>\n\n"
            else:
                prompt += "\n"    
        return prompt

In [20]:
if REASONING:
    tokenizer_file_path = f"Qwen3-{CHOOSE_MODEL}/tokenizer.json"
else:
    tokenizer_file_path = f"Qwen3-{CHOOSE_MODEL}-Base/tokenizer.json"

tokenizer = Qwen3Tokenizer(
    tokenizer_file_path=tokenizer_file_path,
    repo_id=None,
    add_generation_prompt=REASONING,
    add_thinking=REASONING
)

# 5. Generate text

In [24]:
prompt = "Опиши, как работает chain of thoughts"

input_token_ids = tokenizer.encode(prompt)
text = tokenizer.decode(input_token_ids)
text

'<|im_start|>user\nОпиши, как работает chain of thoughts<|im_end|>\n<|im_start|>assistant\n'

In [25]:
# Identical function from chapter 5

def generate(model, idx, max_new_tokens, context_size, temperature=0.2, top_k=None, eos_id=None):
    # For-loop is the same as before: Get logits, and only focus on last time step
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]

        # Filter logits with top_k sampling
        if top_k is not None:
            # Keep only top_k values
            top_logits, _ = torch.topk(logits, top_k)
            min_val = top_logits[:, -1]
            logits = torch.where(logits < min_val, torch.tensor(-torch.inf).to(logits.device), logits)

        # Apply temperature scaling
        if temperature > 0.0:
            logits = logits / temperature

            # Apply softmax to get probabilities
            probs = torch.softmax(logits, dim=-1)  # (batch_size, context_len)

            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (batch_size, 1)

        # Otherwise same as before: get idx of the vocab entry with the highest logits value
        else:
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch_size, 1)

        if eos_id is not None and idx_next.item() == eos_id:
            break  # Stop generating early if end-of-sequence token is encountered and eos_id is specified

        # Same as before: append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch_size, num_tokens+1)

    return idx

In [30]:
import time

torch.manual_seed(21)

start = time.time()

output_token_ids = generate(
    model=model,
    idx=torch.tensor(input_token_ids, device=device).unsqueeze(0),
    max_new_tokens=512,
    context_size=QWEN3_CONFIG["context_length"],
    top_k=1,
    temperature=0.01
)

print(f"Time: {time.time() - start:.2f} sec")

if torch.cuda.is_available():
    max_mem_bytes = torch.cuda.max_memory_allocated()
    max_mem_gb = max_mem_bytes / (1024 ** 3)
    print(f"Max memory allocated: {max_mem_gb:.2f} GB")

output_text = tokenizer.decode(output_token_ids.squeeze(0).tolist())

print(output_text + "...")

Time: 25.36 sec
Max memory allocated: 3.15 GB
<|im_start|>user
Опиши, как работает chain of thoughts<|im_end|>
<|im_start|>assistant
<think>
Хорошо, пользователь спрашивает о том, как работает chain of thought. Нужно объяснить, что это такое, но сначала понять, что именно он хочет. Chain of thought — это логическая последовательность мысли, которая помогает в решении задач. Возможно, он хочет понять, как это применяется в разных областях, или как это связано с алгоритмами.

Надо начать с определения chain of thought. Может, стоит уточнить, что оно включает в себя логические шаги, шаги в решении задачи. Затем объяснить, как это работает, с примерами. Например, в математике, когда решают уравнения, мы следуем шагам. В программировании, когда мы кодируем, мы следуем шагам. В логике, когда мы разбираем задачи.

Может, стоит добавить, что chain of thought помогает в том, чтобы увидеть все возможные варианты, и выбрать наиболее подходящий. Также важно упомянуть, что это не только для задач, 

&nbsp;
# Выводы

Получилось реализовать LLM с chain of thoughts для более глубокого понимания внутренего устройства LLM моделей такого типа.